In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
os.chdir(r"C:\Users\Capta\Documents\GitHub\csgo_econ_viz")

In [4]:
econ_results_merged = pd.read_csv("data/raw/econ_results_merged.csv")

C:\Users\Capta\AppData\Local\Temp\ipykernel_8400\318245247.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  econ_results_merged = pd.read_csv("data/raw/econ_results_merged.csv")


In [ ]:
t1_cols = [f"{i}_t1" for i in range(1, 31)]
t2_cols = [f"{i}_t2" for i in range(1, 31)]
round_cols = [f"{i}_winner" for i in range(1, 31)]

In [6]:
team = "Natus Vincere"
team_t1 = econ_results_merged.query("team_1_x == @team")
team_t2 = econ_results_merged.query("team_2_x == @team")
team_t1_econ = team_t1.loc[:, t1_cols]
team_t2_econ = team_t2.loc[:, t2_cols]
team_t1_econ.columns = team_t1_econ.columns.str.removesuffix("_t1")
team_t2_econ.columns = team_t2_econ.columns.str.removesuffix("_t2")
team_econ = pd.concat([team_t1_econ, team_t2_econ],ignore_index=True)
team_econ.head()

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
0,4150.0,18050.0,21000.0,25850.0,25000.0,25000.0,27250.0,26150.0,26300.0,27850.0,...,23350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4200.0,10000.0,22000.0,24500.0,27550.0,29350.0,31950.0,31850.0,31750.0,32850.0,...,25900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3550.0,18500.0,3500.0,2900.0,23200.0,24600.0,23650.0,23500.0,17150.0,28800.0,...,30100.0,30700.0,26750.0,30500.0,29550.0,28050.0,30850.0,NaN,NaN,NaN
3,4200.0,20200.0,11500.0,1700.0,22550.0,23850.0,24450.0,21250.0,27650.0,31650.0,...,21800.0,23900.0,24950.0,26350.0,27050.0,11850.0,20850.0,3100.0,26000.0,NaN
4,4200.0,10100.0,25900.0,29300.0,30550.0,6750.0,26100.0,8250.0,29600.0,30800.0,...,18050.0,20350.0,27300.0,23700.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

pistol_cols = {1, 16, "1", "16"}
is_pistol_col = team_econ.columns.isin(pistol_cols)

is_pistol = pd.DataFrame(
    np.tile(is_pistol_col, (len(team_econ), 1)),
    index=team_econ.index,
    columns=team_econ.columns
)

avg_spend = team_econ / 5

classifiers = pd.DataFrame(
    np.select(
        condlist=[
            is_pistol,                                    
            avg_spend == 0,                               
            (avg_spend > 0) & (avg_spend < 1500),          
            (avg_spend >= 1500) & (avg_spend < 3500),      
            avg_spend >= 3500        
        ],
        choicelist=["pistol", "hard_save", "eco", "force", "full"],
        default= "N/A"
    ),
    index=team_econ.index, columns=team_econ.columns
)
classifiers.head()

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
0,pistol,full,full,full,full,full,full,full,full,full,...,full,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,pistol,force,full,full,full,full,full,full,full,full,...,full,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,pistol,full,eco,eco,full,full,full,full,force,full,...,full,full,full,full,full,full,full,N/A,N/A,N/A
3,pistol,full,force,eco,full,full,full,full,full,full,...,full,full,full,full,full,force,full,eco,full,N/A
4,pistol,force,full,full,full,eco,full,force,full,full,...,full,full,full,full,N/A,N/A,N/A,N/A,N/A,N/A


In [ ]:
team_t1_round_results = team_t1.loc[:, round_cols].replace({1.0: "W", 2.0: "L"})
team_t2_round_results = team_t2.loc[:, round_cols].replace({2.0: "W", 1.0: "L"})
team_rounds = pd.concat([team_t1_round_results, team_t2_round_results],ignore_index=True)
team_rounds.head()-